In [ ]:
import popjym
import jax
from matplotlib import pyplot as plt
import jax.numpy as jnp
from jax import lax

with jax.log_compiles(True):
    seed = jax.random.PRNGKey(0)
    env, env_params = popjym.make("BattleShipEasy")
    obs, state = env.reset(seed, env_params)
    canva_color = jnp.array([1, 1, 1])
    render = popjym.make_render("BattleShipRender", partial_observable=False, canva_color=canva_color)
    reward = 0
    score = 0
    shape = render.render(state)
    plt.axis('off')
    plt.imshow(shape)
    plt.show()
    for i in range(10):
        
        rng, rng_act, rng_step = jax.random.split(seed, 3)
        action = env.action_space(env_params).sample(rng_act)
        obs2, new_state, reward, term, _ = env.step(rng_step, state, action=action)
        score = lax.cond(reward > 0, lambda _: score + 1, lambda _: score, operand=None)
        state = new_state
        shape = render.render(new_state)
        plt.axis('off')
        plt.imshow(shape)
        plt.show()


In [ ]:
import popjym
import jax
from matplotlib import pyplot as plt
import jax.numpy as jnp
from jax import lax

seed = jax.random.PRNGKey(0)
env, env_params = popjym.make("CartPole")
obs, state = env.reset(seed, env_params)
canva_color = jnp.array([1, 1, 0])
render = popjym.make_render("CartPoleRender", partial_observable=False, canva_color=canva_color)
reward = 0
score = 0
shape = render.render(state, state.score)
plt.axis('off')
plt.imshow(shape)
plt.show()
action = [5, 2, 3, 4, 5]
for i in range(10):

    rng, rng_act, rng_step = jax.random.split(seed, 3)
    action = env.action_space(env_params).sample(rng_act)
    obs2, new_state, reward, term, _ = env.step(rng_step, state, action=action)
    print(reward)
    if term:
        break
    state = new_state
    shape = render.render(state, state.score)
    plt.axis('off')
    plt.imshow(shape)
    plt.show()
    

In [6]:
import matplotlib.pyplot as plt #Alright, I lied, we also need matplotlib as the dependency...
from IPython.display import clear_output
import jax.random as jr
import popjym
import time
from popjym.ppo_equinox.ppo import train
from popjym.ppo_equinox.dataclasses import HyperParameters
from popjym.ppo_equinox.wrappers import GymnaxToBraxWrapper, Agent, ObservationNormalizingWrapper, ActionTanhConstraintWrapper

xdata, ydata = [], []


def progress(step_num, metrics):
    xdata.append(step_num)
    ydata.append(metrics["eval/episode_reward"])
    clear_output(wait=True)

    plt.xlim([0, 40_000_000])
    plt.ylim([0, 4000])
    plt.xlabel("# environment steps")
    plt.ylabel("reward per episode")

    plt.plot(xdata, ydata, "b")
    plt.show()


t0 = time.time()

env, env_params = popjym.make("CartPole")
env = GymnaxToBraxWrapper(env)
env_render = popjym.make_render("CartPoleRender", partial_observable=False)

agent = Agent(jr.PRNGKey(42), 512, 5)
agent = ObservationNormalizingWrapper(agent)
agent = ActionTanhConstraintWrapper(agent, range_low=-1.0, range_high=1.0)

agent, metrics = train(
    agent,
    params=HyperParameters(
        env=env,
        num_timesteps=40_000_000,
        num_evals=10,
        reward_scaling=0.1,
        episode_length=1000,
        unroll_length=5,
        num_minibatches=32,
        num_updates_per_batch=4,
        discounting=0.99,
        learning_rate=3e-4,
        entropy_cost=1e-3,
        batch_size=1024,
        eval_batch_size=256,
        gae_lambda=0.97,
        max_gradient_norm=1.0,
        clipping_epsilon=0.3,
    ),
    env_render=env_render,
    progress=progress,
)

print(f"Training (and tracing) done in {time.time() - t0:.2f} seconds")

AttributeError: 'State' object has no attribute 'pipeline_state'